In [2]:
import cv2
import numpy as np
import math
import os

# =========================================
# --- KONFIGURACJA --- To jest mój kod do pracy inż
# =========================================

BASE_PATH = 'baza_do_porownania'
N_FEATURES = 3000
MATCH_RATIO = 0.8
MIN_SCORE = 30.0
DEBUG = True

# =========================================
# --- FUNKCJE ORB ---
# =========================================

def load_orb_database(folder_path):
    """Wczytuje wszystkie obrazy z podanego folderu i generuje deskryptory ORB."""
    orb = cv2.ORB_create(nfeatures=N_FEATURES)
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    des_list, class_names, img_list = [], [], []

    if not os.path.exists(folder_path):
        if DEBUG:
            print(f"[WARN] Brak folderu: {folder_path}")
        return orb, bf, des_list, class_names, img_list

    for file in os.listdir(folder_path):
        if not file.lower().endswith(('.jpg', '.png', '.jpeg')):
            continue
        img = cv2.imread(os.path.join(folder_path, file), 0)
        kp, des = orb.detectAndCompute(img, None)
        if des is not None:
            des_list.append(des)
            class_names.append(os.path.splitext(file)[0])
            img_list.append(img)

    if DEBUG:
        print(f"[INFO] Wczytano {len(class_names)} wzorców z {folder_path}")
    return orb, bf, des_list, class_names, img_list


def orb_match(roi, orb, bf, des_list, class_names, img_list):
    """Porównuje ROI z bazą znaków z wybranego folderu."""
    if len(des_list) == 0:
        return None, 0, None

    gray_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    kp2, des2 = orb.detectAndCompute(gray_roi, None)
    if des2 is None:
        return None, 0, None

    best_score = 0
    best_name = None
    best_vis = None

    for i, des in enumerate(des_list):
        matches = bf.match(des, des2)
        if not matches:
            continue
        matches = sorted(matches, key=lambda x: x.distance)
        good_matches = [m for m in matches if m.distance < MATCH_RATIO * 100]

        avg_distance = np.mean([m.distance for m in good_matches]) if good_matches else 100
        score = max(0, 100 - avg_distance)

        if score > best_score:
            best_score = score
            best_name = class_names[i]
            best_vis = cv2.drawMatches(
                img_list[i], orb.detectAndCompute(img_list[i], None)[0],
                gray_roi, kp2, matches[:20], None,
                flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
            )

    if DEBUG and best_vis is not None:
        cv2.imshow("Porownanie ORB", best_vis)

    return best_name, best_score, gray_roi


# =========================================
# --- FUNKCJE KOLOR + KSZTAŁT ---
# =========================================

def hsv_frame_process(frame):
    return cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

def color_name_from_hue(hue):
    if (hue <= 5) or (hue >= 170):
        return "Czerwony"
    if 5 < hue < 30:
        return "Pomaranczowy"
    if 85 <= hue < 125:
        return "Niebieski"
    return "Inny"

def color_checking_area(hsv_frame, contour, x, y, w, h):
    H, W = hsv_frame.shape[:2]
    x0, y0 = max(0, int(x)), max(0, int(y))
    x1, y1 = min(W, int(x + w)), min(H, int(y + h))
    if x0 >= x1 or y0 >= y1:
        return None, 0.0

    roi_hsv = hsv_frame[y0:y1, x0:x1]
    contour_pts = contour.reshape(-1, 2).astype(np.int32)
    shifted = contour_pts - np.array([x0, y0])
    mask = np.zeros((y1 - y0, x1 - x0), dtype=np.uint8)
    cv2.drawContours(mask, [shifted], -1, 255, thickness=-1)
    hue_channel = roi_hsv[:, :, 0]
    selected_hues = hue_channel[mask == 255]
    if selected_hues.size == 0:
        return None, 0.0
    unique_hues, counts = np.unique(selected_hues, return_counts=True)
    name_counts = {}
    for u, c in zip(unique_hues, counts):
        name = color_name_from_hue(int(u))
        name_counts[name] = name_counts.get(name, 0) + int(c)
    dominant_name = max(name_counts, key=name_counts.get)
    fraction = name_counts[dominant_name] / selected_hues.size
    return dominant_name, fraction


# =========================================
# --- KLASYFIKATOR ZNAKÓW ---
# =========================================

def trafficsign_classifier(shape_name, dominant_color, roi):
    """Sprawdza kształt i kolor, wybiera odpowiednią bazę i zwraca dopasowanie ORB."""
    
    # Dobór folderu na podstawie kształtu i koloru
    if shape_name == "Osmiokat" and dominant_color == "Czerwony":
        group_folder = os.path.join(BASE_PATH, "STOP")
    elif shape_name == "Trojkat" and dominant_color == "Pomaranczowy":
        group_folder = os.path.join(BASE_PATH, "OSTRZEGAWCZE")
    elif shape_name == "Kolo" and dominant_color != "Niebieski":
        group_folder = os.path.join(BASE_PATH, "OGRANICZENIA")
    elif shape_name == "Kolo" and dominant_color == "Niebieski":
        print(f"[DEBUG] Shape: {shape_name}, Dominant color: {dominant_color}")
        group_folder = os.path.join(BASE_PATH, "NAKAZ")
    else:
        if DEBUG:
            print("[INFO] Niezidentyfikowany znak.")
        return "Niezidentyfikowany", 0

    if DEBUG:
        print(f"[INFO] Klasyfikacja: {shape_name}, kolor: {dominant_color}, folder: {group_folder}")

    orb, bf, des_list, class_names, img_list = load_orb_database(group_folder)
    name, score, _ = orb_match(roi, orb, bf, des_list, class_names, img_list)
    return name, score


# =========================================
# --- DETEKCJA KSZTAŁTÓW ---
# =========================================

def detect_shapes(frame):
    clahe = cv2.createCLAHE(clipLimit=3, tileGridSize=(8, 8))
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    equalized = clahe.apply(gray)
    blur = cv2.GaussianBlur(equalized, (5, 5), 0)
    edges = cv2.Canny(blur, 80, 170)

    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    output = frame.copy()
    hsv_frame = hsv_frame_process(frame)

    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area < 600:
            continue
            
        perimeter = cv2.arcLength(cnt, True)
        if perimeter == 0:
            continue

        print(f"Area: {area}")
        approx = cv2.approxPolyDP(cnt, 0.012 * perimeter, True)
        vertices = len(approx)
        circularity = 4 * math.pi * area / (perimeter * perimeter)
        x, y, w, h = cv2.boundingRect(approx)

        shape_name = None
        color_draw = (0, 255, 0)

        if vertices <= 6 and circularity < 0.7:
            shape_name = "Trojkat"
            color_draw = (0, 255, 255)
        elif vertices > 8 and circularity > 0.85:
            aspect_ratio = float(w) / h if h != 0 else float('inf')
            if 0.7 < aspect_ratio < 1.5:
                shape_name = "Kolo"
                color_draw = (255, 0, 0)
        elif vertices == 8:
            aspect_ratio = float(w) / h if h != 0 else float('inf')
            if 0.9 < aspect_ratio < 1.1:
                shape_name = "Osmiokat"
                color_draw = (255, 0, 255)

        if shape_name:
            dominant_color, fraction = color_checking_area(hsv_frame, approx, x, y, w, h)
            if not dominant_color:
                continue

            roi = frame[y:y+h, x:x+w]
            if roi.size == 0:
                continue

            if DEBUG:
                cv2.imshow("ROI", roi)

            name, score = trafficsign_classifier(shape_name, dominant_color, roi)
            if name and score >= MIN_SCORE:
                label = f"{name} ({score:.0f}%)"
                cv2.drawContours(output, [approx], -1, color_draw, 2)
                cv2.putText(output, label, (x, y - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, color_draw, 2)

    return output, edges, blur


# =========================================
# --- GŁÓWNA PĘTLA ---
# =========================================

def main():
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        output, edges, blur = detect_shapes(frame)
        cv2.imshow("Krawedzie", edges)
        cv2.imshow("Rozmycie", blur)
        cv2.imshow("Wykrywanie znakow", output)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


if __name__ == "__main__":
    main()


Area: 1511.5
Area: 600.0
Area: 1570.0
Area: 623.5
Area: 1182.5
Area: 617.0
Area: 605.5
Area: 643.0
Area: 605.5
Area: 829.5
Area: 829.5
Area: 732.0
Area: 795.0
Area: 635.0
Area: 683.0
Area: 769.5
[INFO] Klasyfikacja: Kolo, kolor: Inny, folder: baza_do_porownania\OGRANICZENIA
[INFO] Wczytano 1 wzorców z baza_do_porownania\OGRANICZENIA
Area: 12930.5
[INFO] Niezidentyfikowany znak.
Area: 12930.0
[INFO] Niezidentyfikowany znak.
Area: 12844.5
[INFO] Niezidentyfikowany znak.
Area: 12596.0
[INFO] Niezidentyfikowany znak.
Area: 12609.5
[INFO] Niezidentyfikowany znak.
Area: 12592.0
[INFO] Niezidentyfikowany znak.
Area: 12594.0
[INFO] Niezidentyfikowany znak.
Area: 12567.5
[INFO] Niezidentyfikowany znak.
Area: 12553.5
[INFO] Niezidentyfikowany znak.
Area: 12533.0
[INFO] Niezidentyfikowany znak.
Area: 12526.0
[INFO] Niezidentyfikowany znak.
Area: 12683.5
[INFO] Niezidentyfikowany znak.
Area: 12706.0
[INFO] Niezidentyfikowany znak.
Area: 12707.5
[INFO] Niezidentyfikowany znak.
Area: 12707.5
[INFO] 

In [ ]:
import cv2
import numpy as np

# === Tworzymy okno i suwaki ===
cv2.namedWindow("Color_Picker")
cv2.resizeWindow("Color_Picker", 600, 300)

# Suwaki HSV
cv2.createTrackbar("H Min", "Color_Picker", 0, 179, lambda x: None)
cv2.createTrackbar("H Max", "Color_Picker", 179, 179, lambda x: None)
cv2.createTrackbar("S Min", "Color_Picker", 0, 255, lambda x: None)
cv2.createTrackbar("S Max", "Color_Picker", 255, 255, lambda x: None)
cv2.createTrackbar("V Min", "Color_Picker", 0, 255, lambda x: None)
cv2.createTrackbar("V Max", "Color_Picker", 255, 255, lambda x: None)

def thresholding(img, h_min, s_min, v_min, h_max, s_max, v_max):
    """Zwraca maskę dla wybranego zakresu HSV."""
    imgHsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    lowerWhite = np.array([h_min, s_min, v_min])
    upperWhite = np.array([h_max, s_max, v_max])
    maskWhite = cv2.inRange(imgHsv, lowerWhite, upperWhite)
    return maskWhite

def getLaneCurve(img, h_min, s_min, v_min, h_max, s_max, v_max):
    """Zwraca maskę progowania i ją pokazuje."""
    imgThresh = thresholding(img, h_min, s_min, v_min, h_max, s_max, v_max)
    cv2.imshow('Thresh', imgThresh)
    return None

# === Główna pętla ===
img = cv2.imread('Linia_drogi/droga2.png')
img = cv2.resize(img, (480, 240))

h_min = 0
s_min = 0
v_min = 150
h_max = 179
s_max = 255
v_max = 255

while True:
    # Pobieramy wartości z suwaków
    h_min = cv2.getTrackbarPos("H Min", "Color_Picker")
    s_min = cv2.getTrackbarPos("S Min", "Color_Picker")
    v_min = cv2.getTrackbarPos("V Min", "Color_Picker")
    h_max = cv2.getTrackbarPos("H Max", "Color_Picker")
    s_max = cv2.getTrackbarPos("S Max", "Color_Picker")
    v_max = cv2.getTrackbarPos("V Max", "Color_Picker")

    # Przetwarzanie i wyświetlanie
    getLaneCurve(img, h_min, s_min, v_min, h_max, s_max, v_max)
    cv2.imshow('Original', img)

    # Przerwij po naciśnięciu 'q'
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np

def nothing(a):
    pass

def initializeTrackbars(initialTracbarVals,wT=480,hT=240):
    """Inicjalizuje okno i suwaki do ustawiania punktów perspektywy."""
    cv2.namedWindow("Points_Setup")
    cv2.resizeWindow("Points_Setup", 600, 300)

    # Suwaki HSV
    cv2.createTrackbar("Width Top", "Points_Setup", initialTracbarVals[0], wT//2, nothing)
    cv2.createTrackbar("Width Bottom", "Points_Setup", initialTracbarVals[1], wT//2, nothing)
    cv2.createTrackbar("Height Top", "Points_Setup", initialTracbarVals[2], hT//2, nothing)
    cv2.createTrackbar("Height Bottom", "Points_Setup", initialTracbarVals[3], hT//2, nothing)

def getTrackbarValues():
    """Zwraca wartości suwaków."""
    width_top = cv2.getTrackbarPos("Width Top", "Points_Setup")
    width_bottom = cv2.getTrackbarPos("Width Bottom", "Points_Setup")
    height_top = cv2.getTrackbarPos("Height Top", "Points_Setup")
    height_bottom = cv2.getTrackbarPos("Height Bottom", "Points_Setup")
    points = np.float32([[width_top, height_top], [width_bottom, height_top], [width_top, height_bottom], [width_bottom, height_bottom]])
    return points

DEBUG = True
h_min = 0
s_min = 0
v_min = 150
h_max = 179
s_max = 255
v_max = 255

def warpIMG(img, points, w, h):
    """Zwraca obraz po perspektywicznym przekształceniu."""
    pts1 = np.float32(points)
    pts2 = np.float32([[0, 0], [w, 0], [0, h], [w, h]])
    matrix = cv2.getPerspectiveTransform(pts1, pts2)
    imgWarp = cv2.warpPerspective(img, matrix, (w, h))
    
    return imgWarp

def thresholding(img):
    """Zwraca maskę dla wybranego zakresu HSV."""
    imgHsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    lowerWhite = np.array([h_min, s_min, v_min])
    upperWhite = np.array([h_max, s_max, v_max])
    maskWhite = cv2.inRange(imgHsv, lowerWhite, upperWhite)
    return maskWhite

def getLaneCurve(img):
    """Zwraca maskę progowania i ją pokazuje."""
    h, w, c, = img.shape
    points = getTrackbarValues
    imgThresh = thresholding(img)
    imgWarp = warpIMG(img, points, w, h)
    cv2.imshow('Thresh', imgThresh)
    cv2.imshow('Warped', imgWarp)
    return None

# === Główna pętla ===
img = cv2.imread('Linia_drogi/droga2.png')
img = cv2.resize(img, (480, 240))
if DEBUG:
    initializeTrackbarsValues = ([100, 100, 100, 100])
    initializeTrackbars(initializeTrackbarsValues, wT=480, hT=240)


while True:


    # Przetwarzanie i wyświetlanie
    getLaneCurve(img)
    cv2.imshow('Original', img)

    # Przerwij po naciśnięciu 'q'
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()


In [ ]:
# ====================== Główny kod do wykrwania środka pasa ======================
import cv2
import numpy as np
# ====================== PROGOWANIE ======================
DEBUG = True
DISPLAY = True
h_min = 0
s_min = 0
v_min = 150
h_max = 179
s_max = 255
v_max = 255

def nothing(a):
    pass

def getHistogram(img, minPer=0.1, display=False):
    """
    Zalecana metoda:
    - sumuje piksele po kolumnach,
    - znajduje najwiekszy pik w lewej polowie i w prawej polowie,
    - sredni punkt = (left_peak + right_peak) // 2 => center of lane.
    Zwraca basePoint i opcjonalny obraz histogramu.
    """
    # img: maska binary (0 lub 255) lub grayscale
    hisValues = np.sum(img, axis=0)  # suma po wierszach -> jedno wejscie na kolumne
    # left / right separation
    midpoint = hisValues.shape[0] // 2
    left_half = hisValues[:midpoint]
    right_half = hisValues[midpoint:]

    # zabezpieczenia: jeśli brak jasnych pikow, fallback na srodek obrazu
    if left_half.max() == 0 and right_half.max() == 0:
        basePoint = midpoint
        imgHist = np.zeros((img.shape[0], img.shape[1], 3), np.uint8) if DISPLAY else None
        return basePoint, imgHist

    left_base = np.argmax(left_half) if left_half.max() != 0 else 0
    right_base = np.argmax(right_half) + midpoint if right_half.max() != 0 else hisValues.shape[0]-1

    basePoint = (int(left_base) + int(right_base)) // 2

    imgHist = None
    if DISPLAY:
        imgHist = np.zeros((img.shape[0], img.shape[1], 3), np.uint8)
        # normalizacja do wysokości obrazu
        maxVal = np.max(hisValues) if np.max(hisValues) > 0 else 1
        for x, intensity in enumerate(hisValues):
            h_line = int((intensity / maxVal) * img.shape[0])
            cv2.line(imgHist, (x, img.shape[0]), (x, img.shape[0] - h_line), (255,0,255), 1)
        cv2.circle(imgHist, (basePoint, img.shape[0]), 8, (0,255,255), cv2.FILLED)
        # opcjonalnie zaznacz left_base / right_base
        cv2.circle(imgHist, (left_base, img.shape[0]), 6, (0,255,0), cv2.FILLED)
        cv2.circle(imgHist, (int(right_base), img.shape[0]), 6, (255,0,0), cv2.FILLED)

    return basePoint, imgHist



# ====================== TRACKBARY ======================
def initializeTrackbars(initialTracbarVals, wT=480, hT=240):
    """Inicjalizuje okno i suwaki do ustawiania trapezu do transformacji perspektywy."""
    cv2.namedWindow("Points_Setup")
    cv2.resizeWindow("Points_Setup", 600, 300)
    cv2.createTrackbar("Width Top", "Points_Setup", initialTracbarVals[0], wT//2, nothing)
    cv2.createTrackbar("Width Bottom", "Points_Setup", initialTracbarVals[1], wT//2, nothing)
    cv2.createTrackbar("Height Top", "Points_Setup", initialTracbarVals[2], hT, nothing)
    cv2.createTrackbar("Height Bottom", "Points_Setup", initialTracbarVals[3], hT, nothing)


def getTrackbarValues(wT=480, hT=240):
    """Zwraca cztery punkty (TL, TR, BL, BR) w formacie wymaganym przez OpenCV."""
    widthTop = cv2.getTrackbarPos("Width Top", "Points_Setup")
    widthBottom = cv2.getTrackbarPos("Width Bottom", "Points_Setup")
    heightTop = cv2.getTrackbarPos("Height Top", "Points_Setup")
    heightBottom = cv2.getTrackbarPos("Height Bottom", "Points_Setup")

    # Tworzymy trapez symetryczny względem środka obrazu
    points = np.float32([
        [wT//2 - widthTop, heightTop],       # Top-Left
        [wT//2 + widthTop, heightTop],       # Top-Right
        [wT//2 - widthBottom, heightBottom], # Bottom-Left
        [wT//2 + widthBottom, heightBottom]  # Bottom-Right
    ])
    return points

def drawPoints(img, points):
    """Rysuje punkty i trapez poprawnie, bez krzyżowania linii."""
    reorder = np.array([0, 1, 3, 2])  # TL, TR, BR, BL zamiast TL, TR, BL, BR
    pts = np.int32(points[reorder])
    for x, y in pts:
        cv2.circle(img, (int(x), int(y)), 15, (0, 0, 255), cv2.FILLED)
    cv2.polylines(img, [pts], isClosed=True, color=(0, 255, 0), thickness=2)
    return img

# ====================== TRANSFORMACJA ======================
def warpIMG(img, points, w, h):
    """Zwraca obraz po perspektywicznym przekształceniu."""
    pts1 = np.float32(points)
    pts2 = np.float32([[0, 0], [w, 0], [0, h], [w, h]])
    matrix = cv2.getPerspectiveTransform(pts1, pts2)
    imgWarp = cv2.warpPerspective(img, matrix, (w, h))
    return imgWarp




def thresholding(img):
    """Zwraca maskę dla wybranego zakresu HSV."""
    imgHsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    lowerWhite = np.array([h_min, s_min, v_min])
    upperWhite = np.array([h_max, s_max, v_max])
    maskWhite = cv2.inRange(imgHsv, lowerWhite, upperWhite)
    return maskWhite


# ====================== GŁÓWNA FUNKCJA ======================
def getLaneCurve(img):
    """Zwraca maskę progowania i wynik transformacji perspektywicznej."""
    h, w, c = img.shape
    points = getTrackbarValues(w, h)
    imgThresh = thresholding(img)
    imgWarp = warpIMG(imgThresh, points, w, h)

    imgWarpPoints = drawPoints(img.copy(), points)

    basePoint, imgHist = getHistogram(imgWarp, display=DISPLAY)
    # Podgląd
    cv2.circle(imgWarpPoints, (basePoint, img.shape[0]), 15, (0,255,255), cv2.FILLED)
    cv2.imshow('Thresh', imgThresh)
    cv2.imshow('Warped', imgWarp)
    cv2.imshow('Warped Points', imgWarpPoints)
    cv2.imshow('Histogram', imgHist)
    return None


# ====================== MAIN ======================
img = cv2.imread('Linia_drogi/droga3.png')
img = cv2.resize(img, (480, 240))

if DEBUG:
    initializeTrackbarsValues = [140, 240, 116, 240]  # początkowe wartości suwaków
    initializeTrackbars(initializeTrackbarsValues, wT=480, hT=240)

while True:
    getLaneCurve(img)
    cv2.imshow('Original', img)

    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()


In [2]:
# ====================== Główny kod do wykrwania środka pasa ======================
import cv2
import numpy as np
# ====================== PROGOWANIE ======================
DEBUG = True
DISPLAY = True
h_min = 0
s_min = 0
v_min = 150
h_max = 179
s_max = 255
v_max = 255
curveList = []
CURVELIST_LENGTH = 10
def nothing(a):
    pass

def getHistogram(img, minPer=0.1, display=None, region=1):
    """
    Wyznacza środek pasa z histogramu kolumnowego.
    - region == 1: prognoza (uśrednianie / centroid z całego obrazu) -- stablilniejsze przewidywanie
    - region != 1: aktualny środek (argmax w dolnym regionie) -- dokładniejszy dla bieżącej pozycji
    display: jeśli None -> użyje globalnej zmiennej DISPLAY, jeśli True -> zwróci również obraz histogramu
    """
    # Jeśli nie przekazano jawnie display, użyj globalnej wartości DISPLAY
    if display is None:
        display = DISPLAY

    h, w = img.shape[:2]

    # Wybierz ROI w zależności od regionu:
    if region == 1:
        roi = img
    else:
        start_row = h - h // region
        roi = img[start_row:, :]

    # Histogram: suma wartości pikseli po kolumnach
    hisValues = np.sum(roi, axis=0)
    midpoint = w // 2
    left_half = hisValues[:midpoint]
    right_half = hisValues[midpoint:]

    # Jeśli brak danych po którejkolwiek stronie -> fallback na środek obrazu
    if left_half.max() == 0 or right_half.max() == 0:
        basePoint = midpoint
        imgHist = np.zeros((h, w, 3), np.uint8) if display else None
        return basePoint, imgHist

    # Dwa tryby: prognoza (region==1) -> uśrednianie (centroid ważony intensywnością)
    # i aktualny (region!=1) -> argmax (maksimum pików)
    if region == 1:
        # prognoza: wybieramy indeksy przekraczające threshold i liczymy centroid (ważony)
        thr_left = minPer * (left_half.max() if left_half.max() > 0 else 1)
        idxs_l = np.where(left_half >= thr_left)[0]
        if idxs_l.size == 0:
            left_mean = np.argmax(left_half)
        else:
            weights_l = left_half[idxs_l].astype(np.float64)
            left_mean = np.average(idxs_l, weights=weights_l)

        thr_right = minPer * (right_half.max() if right_half.max() > 0 else 1)
        idxs_r = np.where(right_half >= thr_right)[0]
        if idxs_r.size == 0:
            right_mean = np.argmax(right_half) + midpoint
        else:
            weights_r = right_half[idxs_r].astype(np.float64)
            right_mean = np.average(idxs_r, weights=weights_r) + midpoint
    else:
        # aktualny środek: bierzemy najsilniejsze piki (dokładność lokalna)
        left_mean = np.argmax(left_half)
        right_mean = np.argmax(right_half) + midpoint

    # wynikowy punkt (środek między wykrytymi krawędziami)
    basePoint = int((int(round(left_mean)) + int(round(right_mean))) // 2)

    # Rysowanie histogramu (opcjonalnie)
    imgHist = None
    if display:
        imgHist = np.zeros((h, w, 3), np.uint8)
        maxv = hisValues.max() if hisValues.max() > 0 else 1
        for x, intensity in enumerate(hisValues):
            h_line = int((intensity / maxv) * h)
            cv2.line(imgHist, (x, h), (x, h - h_line), (255, 0, 255), 1)

        # zaznacz lewy/prawy wykryty punkt i basePoint
        lx = int(round(left_mean))
        rx = int(round(right_mean))
        cv2.circle(imgHist, (lx, h), 6, (0, 255, 0), cv2.FILLED)      # lewy (zielony)
        cv2.circle(imgHist, (rx, h), 6, (255, 0, 0), cv2.FILLED)      # prawy (niebieski)
        cv2.circle(imgHist, (basePoint, h), 8, (0, 255, 255), cv2.FILLED)  # center (żółty)

    return basePoint, imgHist





# ====================== TRACKBARY ======================
def initializeTrackbars(initialTracbarVals, wT=480, hT=240):
    """Inicjalizuje okno i suwaki do ustawiania trapezu do transformacji perspektywy."""
    cv2.namedWindow("Points_Setup")
    cv2.resizeWindow("Points_Setup", 600, 300)
    cv2.createTrackbar("Width Top", "Points_Setup", initialTracbarVals[0], wT//2, nothing)
    cv2.createTrackbar("Width Bottom", "Points_Setup", initialTracbarVals[1], wT//2, nothing)
    cv2.createTrackbar("Height Top", "Points_Setup", initialTracbarVals[2], hT, nothing)
    cv2.createTrackbar("Height Bottom", "Points_Setup", initialTracbarVals[3], hT, nothing)


def getTrackbarValues(wT=480, hT=240):
    """Zwraca cztery punkty (TL, TR, BL, BR) w formacie wymaganym przez OpenCV."""
    widthTop = cv2.getTrackbarPos("Width Top", "Points_Setup")
    widthBottom = cv2.getTrackbarPos("Width Bottom", "Points_Setup")
    heightTop = cv2.getTrackbarPos("Height Top", "Points_Setup")
    heightBottom = cv2.getTrackbarPos("Height Bottom", "Points_Setup")

    # Tworzymy trapez symetryczny względem środka obrazu
    points = np.float32([
        [wT//2 - widthTop, heightTop],       # Top-Left
        [wT//2 + widthTop, heightTop],       # Top-Right
        [wT//2 - widthBottom, heightBottom], # Bottom-Left
        [wT//2 + widthBottom, heightBottom]  # Bottom-Right
    ])
    return points

def drawPoints(img, points):
    """Rysuje punkty i trapez poprawnie, bez krzyżowania linii."""
    reorder = np.array([0, 1, 3, 2])  # TL, TR, BR, BL zamiast TL, TR, BL, BR
    pts = np.int32(points[reorder])
    for x, y in pts:
        cv2.circle(img, (int(x), int(y)), 15, (0, 0, 255), cv2.FILLED)
    cv2.polylines(img, [pts], isClosed=True, color=(0, 255, 0), thickness=2)
    return img

# ====================== TRANSFORMACJA ======================
def warpIMG(img, points, w, h):
    """Zwraca obraz po perspektywicznym przekształceniu."""
    pts1 = np.float32(points)
    pts2 = np.float32([[0, 0], [w, 0], [0, h], [w, h]])
    matrix = cv2.getPerspectiveTransform(pts1, pts2)
    imgWarp = cv2.warpPerspective(img, matrix, (w, h))
    return imgWarp




def thresholding(img):
    """Zwraca maskę dla wybranego zakresu HSV."""
    imgHsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    lowerWhite = np.array([h_min, s_min, v_min])
    upperWhite = np.array([h_max, s_max, v_max])
    maskWhite = cv2.inRange(imgHsv, lowerWhite, upperWhite)
    maskWhite = cv2.morphologyEx(maskWhite, cv2.MORPH_CLOSE, np.ones((5,5), np.uint8))
    maskWhite = cv2.GaussianBlur(maskWhite, (5,5), 0)
    maskWhite = cv2.ximgproc.thinning(maskWhite)

    return maskWhite


# ====================== GŁÓWNA FUNKCJA ======================
def getLaneCurve(img):
    """Zwraca maskę progowania i wynik transformacji perspektywicznej."""
    h, w, c = img.shape
    points = getTrackbarValues(w, h)
    imgThresh = thresholding(img)
    imgWarp = warpIMG(imgThresh, points, w, h)

    imgWarpPoints = drawPoints(img.copy(), points)

    middlePoint, img_midHist = getHistogram(imgWarp,minPer=0.5, display=DISPLAY, region=4)
    curveAveragePoint, img_CurveHist = getHistogram(imgWarp,minPer=0.5, display=DISPLAY)
    curveRaw = curveAveragePoint-middlePoint
    
    curveList.append(curveRaw)
    if len(curveList) > CURVELIST_LENGTH:
        curveList.pop(0)
    curve = int(sum(curveList)/len(curveList))


    print(curve)
    cv2.circle(imgWarpPoints, (curveAveragePoint, img.shape[0]), 15, (0,255,255), cv2.FILLED)
    cv2.imshow('Warped', imgWarp)
    cv2.imshow('Warped Points', imgWarpPoints)
    cv2.imshow('Mid Histogram', img_midHist)
    cv2.imshow('Curve Histogram', img_CurveHist)
    return None


# ====================== MAIN ======================
img = cv2.imread('Linia_drogi/droga2.png')
img = cv2.resize(img, (480, 240))

if DEBUG:
    initializeTrackbarsValues = [140, 240, 116, 240]  # początkowe wartości suwaków
    initializeTrackbars(initializeTrackbarsValues, wT=480, hT=240)

while True:
    getLaneCurve(img)


    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()


-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-4
-5
-6
-6
-7
-8
-9
-9
-10
-11
-10
-10
-9
-9
-8
-8
-8
-8
-8
-8
-9
-10
-10
-11
-12
-12
-12
-12
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
-13
